In [ ]:
# all imports
import os as os
import pandas as pd
import glob
import csv
from pathlib import Path
import html, re
from w3lib.html import replace_entities
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import gensim
stopWords = set(stopwords.words('english') )
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk import FreqDist
from nltk.collocations import *

In [ ]:
# add some punctuation to string.punctuation
punc = string.punctuation + '“”'

# documents get tagged by an index (number), while filenames have different numbers
# keep track of this
fileIdToIndex = {} # given a fileId -> tag
indexToFileId=[] # given a tag -> fileId

class MyFiles(object):
    def __init__(self, dirname, tokens_only = False):
        self.dirname = dirname
        self.tokens_only = tokens_only
 
    def __iter__(self):
        #for i, fname in enumerate(files[0:200]):
         
        for i, fname in enumerate(os.listdir(self.dirname)[0:4604]):    # there are 4604 files in total
        # enumerate = return a list of tuples, iterate from start to end
        # os.listdir = return index of a directory, input = directory address
        # this part enumerates the first 200 units in the index under dirname
        
            with open( os.path.join(self.dirname, fname), encoding='utf-8') as f:
                content = f.read()
            # filter
                #content = [f for f in content if int(f["length"])> 1000 and (f["date"][0:4]) == '2019']
                #content = [f for f in content if len(f)> 1000]
            # grab id from filename
            myCounter = int ( re.findall(r'(\d*)\.txt', fname)[0] )
            # update 
            fileIdToIndex [ myCounter] = i
            indexToFileId.append( myCounter)
            #print('fname', fname, 'tag', myCounter)
            file_tokens = [x for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation]
            
            if self.tokens_only == True:
                yield file_tokens
            else:
                yield TaggedDocument(words=file_tokens, tags=[i] )


In [ ]:
# Hipergator
ffiles = MyFiles(r'/blue/acg7849/tbode/item1') # this one expected str, bytes or os.PathLike object
#ffiles = MyFiles(r'/blue/acg7849/share/BS/item1/') # a memory-friendly iterator
# dirname = '/blue/acg7849/share/BS/item1/'?

In [ ]:
# create a model, build vocabulary
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=8)
model.build_vocab(ffiles)

In [ ]:
# train it
model.train(ffiles, total_examples=model.corpus_count, epochs=model.epochs)
print('FINISH')

In [ ]:
# Hipergator
def tokenizeFile(file_id):
    with open( r'/blue/acg7849/tbode/item1/'+str(file_id)+'.txt', encoding='utf-8') as f:
            content = f.read()
    return ([x for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation] )

In [ ]:
t = tokenizeFile(277350) # test file = 277350
model.infer_vector( t )

In [ ]:
t = tokenizeFile(277350)
inferred_vector = model.infer_vector( t )
# dv is short for docvecs
# most similar files to 277350 is the 4th one 265065?
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
#sims = model.dv.most_similar([inferred_vector], topn=4)
sims[0:5]

In [ ]:
# letter with filename 1.txt is the first letter, so tag is 0
similar_doc = model.docvecs.most_similar(0)
similar_doc

In [ ]:
print('number of documents', model.corpus_count)
print('model.docvecs', len(model.dv))

In [ ]:
# Hipergator
# reread the files, and get the vector for each file
# feed vector into k-means algorithm to make clusters
wordLists = MyFiles(r'/blue/acg7849/tbode/item1', tokens_only = True) # a memory-friendly iterator
vectors = [ model.infer_vector( w ) for w in wordLists]
print(len(vectors))
vectors[0]

In [ ]:
from nltk.cluster import KMeansClusterer
num_clusters = 50
kclusterer = KMeansClusterer(num_clusters, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(vectors, assign_clusters=True)
assigned_clusters[0:10]

In [ ]:
import collections
print(collections.Counter(assigned_clusters))

### Industrial Codes

In [1]:
# Install Packages

!pip install w3lib first

import html, re
# need to do: pip install w3lib first
from w3lib.html import replace_entities

import csv
import os as os
import pandas as pd
import glob as glob

Defaulting to user installation because normal site-packages is not writeable


In [2]:
mydirectory = '/blue/acg7849/share/10Ks'
file_list = glob.glob(mydirectory + '/*.txt')
#print(file_list)

In [35]:
SIC_list = []
fileName_list = []

for file_path in file_list:
    # read the file
    with open(file_path) as f:
        filing = f.read()

    SICregex = 'CLASSIFICATION:.*?\[(\d{4})'
    SIC = re.findall(SICregex, filing) 
    SIC_list.append(SIC);
        
    #need to input FileName
    shortName = file_path[file_path.rfind('/')+1:]  
    fileName_list.append(shortName);
      
    DF = pd.DataFrame(list(zip(fileName_list, SIC_list)), columns = ['FileName','Standard Industrial Classification'])
   

In [38]:
#DF

In [37]:
DF.to_csv('SIC.csv', index = False)